# Хеширование данных в таблице Excel

## Описание задачи

Рассмотрим хеширование данных на примере таблицы с покупателями:

|Customer            |Name  |Comment|
|--------------------|------|-------|
|ivan@mail.ru        |Иван  |       |
|company@business.com|      |       |
|alexa@yandex.ru     |Саша  |       |

В этой таблице есть поля **Customer**, **Name** и **Comment**. Колонка **Customer** содержит обязательные для идентификации данные — email-адрес.

Хотим хешировать данные и получить таблицу следующего вида:

|Customer            |Name  |Comment|hashed_CustomerName|
|--------------------|------|-------|-------------------|
|ivan@mail.ru        |Иван  ||6ba5e9539184c139f34451cb17b7d3e7d35eaab2f9937b73d1a3aabc899550ae|
|company@business.com|      ||7f4b8deb0c3f9bc96d0335f993dcd42df4835394cf8ff8c478691f6a8c591cdb|
|alexa@yandex.ru     |Саша  ||71f644bd737221da3271eff0edc946501b57cccd1812c7b3b4781ad4160cf72b|

## Подготовка данных

Подключаем библиотеки hashlib и pandas

In [1]:
import hashlib
import pandas as pd

### Обязательные колонки

Указываем через запятую список колонок, которые обязательны для идентификации покупателя. В нашем случае — **Customer**. Строки таблицы, в которых не заполнено хотя бы одно поле из этого списка, будут пропущены.

*Это также могут быть колонки с номером телефона, логином или уникальным id. Таких колонок может быть несколько, но не меньше одной.*

In [2]:
required_columns = ['Customer']

### Дополнительные колонки

Указываем любые другие уникальные колонки, которые хотим включить в хеш. Для примера добавим поле **Name**.

*Оставьте пустой список `[]`, если вам достаточно колонок из первого списка*

In [3]:
additional_columns = ['Name']

### Загрузка данных из файла

Читаем таблицу с покупателями. Данные счтываем как строки. Удаляем строки, если в них есть пустые или нераспознанные значения в обязательных колонках, и заменяем в оставшихся строках нераспознанные и пустые поля на пустые строки.

In [4]:
df_customers = pd.read_excel('customers.xlsx', dtype=str).dropna(subset=required_columns).fillna('')

Можем посмотреть последние 5 строк, чтобы убедиться, что всё прочиталось.

*Для просмотра первых N-строк используйте head(N)*

In [5]:
df_customers.tail(5)

,Customer,Name,Comment
0,ivan@mail.ru,Иван,
1,company@business.com,,
2,alexa@yandex.ru,Саша,


## Хеширование

Это наша хеширующая процедура. Она склеивает все переданные ей колонки в строке, вычисляет хеш sha256, и дописывает к целевому дата-фрейму новую колонку с результатами. Имя колонки будет составлено склейкой из названий колонок, по которым выполняется хеширование.

*Можно выбрать другую хеш-функцию, заменив `hashlib.sha256(x)` на `hashlib.md5(x)` или любую другую, доступную в [hashlib](https://docs.python.org/3/library/hashlib.html)*


In [6]:
def hash_data(source_df, destination_df, *column):
    column_name = ''
    destination_df['hashed_'+column_name.join(column)] = source_df[list(column)].sum(axis=1).str.encode('utf-8').apply(lambda x: (hashlib.sha256(x).hexdigest()))

Вычисляем хеши для указанного ранее списка колонок. Здесь указан один и тот же дата-фрейм для источника и назначения, поэтому колонка с хешами добавится к исходным данным.

In [7]:
hash_data(df_customers, df_customers, *required_columns, *additional_columns)

Проверяем результат

In [8]:
df_customers.tail(5)

,Customer,Name,Comment,hashed_CustomerName
0,ivan@mail.ru,Иван,,6ba5e9539184c139f34451cb17b7d3e7d35eaab2f9937b...
1,company@business.com,,,7f4b8deb0c3f9bc96d0335f993dcd42df4835394cf8ff8...
2,alexa@yandex.ru,Саша,,71f644bd737221da3271eff0edc946501b57cccd1812c7...


## Выгрузка

Сохраняем результат в новую таблицу.

*Если вы получили ошибку, скорее всего вы выгружаете таблицу в файл повторно. Закройте окно Excel, в котором открыт этот файл, и выполните этот шаг ещё раз.*

In [9]:
df_customers.to_excel('hashed_customers.xlsx')